# Query Expansion

0) Just some imports

In [1]:
import re



import math
import numpy as np
import common as cm

# 1) Simple search engine

1.1) Get acquainted with the below class. There are several TODOs. However, DO NOT complete them now. 

In [2]:
class Dictionary:
    def __init__(self):
        ### keeps unique terms (SORTED!)
        self.terms = self.loadTerms("terms.txt")
        self.idfs = [] ### IDF coefficients
        self.corM = [] ### a correlation matrix

    ### load terms
    def loadTerms(self, fileName):
        file = open(fileName,'r', encoding='utf-8-sig')
        k = [self.proc(s) for s in file.readlines()]
        k.sort()
        return k

    ### ignore it
    def proc(self, s):
        if s[-1] == '\n': return s[:-1]
        else: return s
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    def computeIDFs(self, documents):
        inDocList = []
        for term in self.terms:
            inDoc = 0
            for document in documents:
                for docTerm in document.tokens:
                    if term == docTerm:
                        inDoc += 1
                        break
            inDocList.append(inDoc)
        self.idfs = [math.log((len(documents)/x), len(documents)) for x in inDocList]
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    def computeCorM(self, documents):
        for i in range(len(self.terms)):
            any = 0
            tab = []
            for j in range(len(documents)):
                tab.append(0);
                tab[j] = len([term for term in documents[j].tokens if term == self.terms[i]])
            self.corM.append(tab)
        for i in range(len(self.corM)):
            sum = 0
            for j in range(len(self.corM[i])):
                sum += math.pow(self.corM[i][j], 2)
            sum = math.sqrt(sum)
            for j in range(len(self.corM[i])):
                self.corM[i][j] /= sum
        transM = np.transpose(self.corM)
        self.corM = np.matmul(self.corM, transM)
        for i in range(len(self.corM)):
            self.corM[i][i] = -1;
        

### SOME DEBUG
dictionary = Dictionary()
print(dictionary.terms[:50])

['aaai', 'about', 'academic', 'access', 'acquired', 'acquisition', 'action', 'activity', 'actual', 'adaptive', 'add', 'advance', 'agricultural', 'aha', 'aim', 'alert', 'algorithm', 'all', 'analysis', 'and', 'announcement', 'answer', 'anyone', 'application', 'applied', 'apply', 'applying', 'approach', 'approache', 'april', 'archive', 'are', 'area', 'areas', 'article', 'artificial', 'asked', 'august', 'author', 'automated', 'automatically', 'autonomous', 'available', 'awards', 'backend', 'backgammon', 'baldi', 'based', 'basic', 'bayesian']


1.2) Load files: here we load some example collection of documents. RAW_DOCUMENTS = just strings. Check if the documents are loaded correctly (e.g., print RAW_DOCUMENTS[0])

In [3]:
RAW_DOCUMENTS = cm.loadDocuments("documents.txt")
### SOME DEBUG
print(RAW_DOCUMENTS[0])

David W. Aha:  Machine Learning Page
 Machine Learning Resources. Suggestions welcome. ... (WizRule); ZDM Scientific
 Ltd. Conference Announcements. Courses on Machine Learning. Data Repositories. ... 
 Description: Comprehensive machine learning resources from Applications to Tutorials.



In [4]:
### SOME DEBUG, JUST RUN; check if (a) common.py is imported correctly and (b) 
### tokens are correctly derived from some document (e.g., RAW_DOCUMENTS[0])
print(cm.simpleTextProcessing(RAW_DOCUMENTS[0], re))

['david', 'aha', 'machine', 'learning', 'page', 'machine', 'learning', 'resource', 'suggestion', 'welcome', 'wizrule', 'zdm', 'scientific', 'ltd', 'conference', 'announcement', 'course', 'machine', 'learning', 'data', 'repository', 'description', 'comprehensive', 'machine', 'learning', 'resource', 'from', 'application', 'tutorials']


1.3) Get acquainted with the below class. 

In [5]:
class Document:
    def __init__(self, doc_id, raw_document, dictionary):
        self.doc_id = doc_id ### DOC ID, simply 0,1,2,3....
        self.raw_document = raw_document ### raw data, i.e., string data
        self.dictionary = dictionary # reference to the dictionary
        
        ### DOCUMENT REPRESENTATIONS
        self.tokens = cm.simpleTextProcessing(raw_document, re) ### get terms
        self.bow = [] # Bag Of Words (BOW - number of term occurences)
        self.tf = [] # TF representation
        self.tf_idf = [] # TF-IDF representation

    ### TODO - complete this method; it should compute a BOW representation
    def computeBOW_Representation(self):
        for term in self.tokens:
            count = 0
            for term2 in self.tokens:
                if term == term2:
                    count += 1
            self.bow.append(count)
    
    ### TODO - complete this method; it should compute a TF representation
    def computeTF_Representation(self):
        self.tf = [(count/max(self.bow)) for count in self.bow]
    
    ### TODO - complete this method; it should compute a TFxIDF representation 
    ### (important: it should not be run before dictionary.idfs are not computed!)
    def computeTF_IDF_Representation(self):
        for i in range(len(self.tf)):
            for j in range(len(self.dictionary.terms)):
                if self.dictionary.terms[j] == self.tokens[i]:
                    self.tf_idf.append(self.tf[i] * self.dictionary.idfs[j])
                    
            
    
    def computeRepresentations(self):
        self.computeBOW_Representation()
        self.computeTF_Representation()
        self.computeTF_IDF_Representation()
    
documents = [Document(i, RAW_DOCUMENTS[i], dictionary) for i in range(len(RAW_DOCUMENTS))]


1.4) Compute IDFs here

In [6]:
### TODO COMPUTE IDFS HERE (FINISH THE PROPER METHOD OF THE DICTIONARY CLASS - DO NOT FORGET TO RE-RUN THE CELL)
dictionary.computeIDFs(documents)

### SOME DEBUG
res = [[dictionary.terms[i], dictionary.idfs[i]] for i in range(len(dictionary.terms))]
res.sort(key = lambda x: x[1])
# LEAST COMMON WORDS - HIGH IDF
print(res[-5:])
# MOST COMMON WORDS - LOW IDF
print(res[:5])

[['working', 1.0], ['www', 1.0], ['york', 1.0], ['young', 1.0], ['zdm', 1.0]]
[['learning', 0.005134641233681233], ['machine', 0.005134641233681233], ['the', 0.10094954712397083], ['and', 0.16520249131202333], ['description', 0.19963159196149574]]


1.5) Compute the document representations (for each document run computeRepresentations())

In [7]:
for d in documents: d.computeRepresentations()
### SOME DEBUG (you should see some 4s - which terms are these?)
print(documents[0].bow)
print('\n')
print(documents[0].tf)
print('\n')
print(documents[0].tf_idf)

[1, 1, 4, 4, 1, 4, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 1, 1, 1, 1, 4, 4, 2, 1, 1, 1]


[0.25, 0.25, 1.0, 1.0, 0.25, 1.0, 1.0, 0.5, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.25, 1.0, 1.0, 0.5, 0.25, 0.25, 0.25]


[0.1725938086321159, 0.1725938086321159, 0.005134641233681233, 0.005134641233681233, 0.0774061913678841, 0.005134641233681233, 0.005134641233681233, 0.24286219817679816, 0.25, 0.1725938086321159, 0.25, 0.25, 0.25, 0.25, 0.1112508533112449, 0.2112969043160579, 0.1725938086321159, 0.005134641233681233, 0.005134641233681233, 0.1112508533112449, 0.188657044679129, 0.049907897990373935, 0.2112969043160579, 0.005134641233681233, 0.005134641233681233, 0.24286219817679816, 0.13389071294817384, 0.14134667383281885, 0.188657044679129]


1.6) Finish the below method. It should compute and return a cosine similarity (v1 and v2 are two vectors - tf-idf representations)

In [8]:
### TODO 
def getSimilarity(v1, v2):
    return (sum([a * b for a, b in zip(v1,v2)]))/(math.sqrt(sum([math.pow(a, 2) for a in v1]) * sum([math.pow(b, 2) for b in v2])))

1.7) Run the below script for different queries. getTopResults seeks for documents being the most similar/relevant to the query. Do you find the results satisfactory?

In [9]:
# query = "machine learning"
#query = "academic research"
#query = "international conference"
query = "international conference washington"

In [10]:
def getTopResults(query, documents, dictionary, similarity, top = 5):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults(query, documents, dictionary, getSimilarity, top = 5)

RANK = 1 WITH SIMILARITY = 0.6646561196491053 | DOC ID = 39
ICML2002 - Sydney
 The Nineteenth International Conference on Machine Learning (ICML-2002). ... Previous
 meetings on machine learning: ICML-2001, ICML-2000, ICML-99 , ICML-98. ... 


RANK = 2 WITH SIMILARITY = 0.5739996933025575 | DOC ID = 76
ICML 2003
 The Twentieth International Conference on Machine Learning (ICML-2003).
 August 21-24, 2003 Washington, DC USA. The Twentieth International ... 


RANK = 3 WITH SIMILARITY = 0.5448792733955077 | DOC ID = 31
New Link To Robert Holte's home page for Machine Learning
 New Link To Robert Holte's home page for Machine Learning. The web
 pages maintained by Robert Holte for Machine Learning have moved. ... 


RANK = 4 WITH SIMILARITY = 0.5249037627063615 | DOC ID = 10
Kluwer
 Kluwer Alert, Subscribe today for our free electronic notification service
 for journal tables of contents and new product announcements ... 


RANK = 5 WITH SIMILARITY = 0.49783648821996174 | DOC ID = 74
ECML/

# 2) Query expansion

## 2.1) Correlation matrix

2.1.1) Finish dictionary.computeCorM method (see class Dictionary). It should generate a correlation matrix (correlation between terms).

IMPORTANT: although corM[ i ][ i ] (for each i) should be 1.0, set it to -1.0

In [11]:
### TODO - COMPLETE THE computeCorM METHOD (see one of the first cells)
dictionary.computeCorM(documents)
print(dictionary.corM)

[[-1.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.         -1.          0.         ...  0.18898224  0.
   0.        ]
 [ 0.          0.         -1.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.18898224  0.         ... -1.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.         -1.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
  -1.        ]]


2.1.2) Finish the below method. For each term in the query (you must parse the query, see getTopResults() method), find another term which is the most correlated with the input term.

In [12]:
# query = "machine"
# query = "algorithm"
# query = "learning"
# query = "conference"
query = "research"
# query = "concept"

def suggestKeywords(query, dictionary):
    ### TODO
    result = ""
    for i in range(len(dictionary.terms)):
        if query == dictionary.terms[i]:
            maxInd = 0
            for j in range(len(dictionary.corM[i])):
                if dictionary.corM[i][j] > dictionary.corM[i][maxInd]: maxInd = j
            result = dictionary.terms[maxInd]
            break;
    print(result)
    pass
        
suggestKeywords(query, dictionary)

group


# 2.2) Rocchio algorithm

$\overrightarrow{q_{m}} = \alpha \overrightarrow{q} + \left(\beta \cdot \dfrac{1}{|D_{r}|} \sum_{\overrightarrow{D_j} \in D_{r}} \overrightarrow{D_j} \right) - \left(\gamma \cdot \dfrac{1}{|D_{nr}|} \sum_{\overrightarrow{D_j} \in D_{nr}} \overrightarrow{D_j} \right)$

2.2.1) Firstly, run the below code. Observe the results. Assume that we do not like the first and the second result (Docs 63 and 77). However, assume that docs 29 and 36 are satisfactory. Now, modfify the method. It should alter the query vector, according to Rocchio algorithm. Check the result for the above considered scenario (relevant docs = 29 and 36; not relevant = 63 and 77). Check the results for different values of alpha, beta, and gamma coefficients. 

In [13]:
def getTopResults_Rocchio(query, 
                          documents, 
                          dictionary, 
                          similarity, 
                          rel_docs = [29, 36],
                          nrel_docs = [63, 77],
                          alpha = 0,
                          beta = 0.8,
                          gamma = 0.2,
                          top = 10):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ##### TODO: MODIFY qd.tf_idf HERE
    rels = []
    for rel in rel_docs:
        tab = []
        for word in qd.tokens:
            print(word)
            relDoc = [document for document in documents if document.doc_id == rel][0]
            relQuery = [relDoc.tf_idf[i] for i in range(len(relDoc.tf_idf)) if (relDoc.tokens[i] == word)]
            value = 0 if len(relQuery) == 0 else relQuery[0]
            tab.append(value)
        rels.append(tab)
        
    nrels = []
    for rel in nrel_docs:
        tab = []
        for word in qd.tokens:
            print(word)
            relDoc = [document for document in documents if document.doc_id == rel][0]
            relQuery = [relDoc.tf_idf[i] for i in range(len(relDoc.tf_idf)) if (relDoc.tokens[i] == word)]
            value = 0 if len(relQuery) == 0 else relQuery[0]
            tab.append(value)
        nrels.append(tab)
    relDocValue = [beta * (value/len(rels)) for value in np.sum(rels, axis=0)]
    nrelDocValue = [gamma * (value/len(nrels)) for value in np.sum(nrels, axis=0)]
    basicValue = [elem * alpha for elem in qd.tf_idf]
    qd.tf_idf = [basicValue[i] + relDocValue[i] - nrelDocValue[i] for i in range(len(basicValue))]
    print("")
    print("")
    #####
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults_Rocchio("machine learning", documents, dictionary, getSimilarity, top = 10)

machine
learning
machine
learning
machine
learning
machine
learning


RANK = 1 WITH SIMILARITY = 0.7256420167563681 | DOC ID = 45
University of York Machine Learning Group
 Content Introduction. 


RANK = 2 WITH SIMILARITY = 0.5587348552355521 | DOC ID = 10
Kluwer
 Kluwer Alert, Subscribe today for our free electronic notification service
 for journal tables of contents and new product announcements ... 


RANK = 3 WITH SIMILARITY = 0.5292518458228739 | DOC ID = 62
Open Directory - Computers: Artificial Intelligence: Machine ... 
 ... David W. Aha: Machine Learning Page - Comprehensive machine learning
 resources from Applications to Tutorials. Machine Learning ... 


RANK = 4 WITH SIMILARITY = 0.5050922063815935 | DOC ID = 82
IJCAI 99 Workshop: Machine Learning for Information Filtering. Workshop
 on "Machine Learning for Information Filtering" at IJCAI 99. ... 


RANK = 5 WITH SIMILARITY = 0.4771068564123384 | DOC ID = 32
IFIP WG 12.2 Global Web Site for Machine Learning and Case-Bas

# 2.3) WordNet

2.3.1) Installation

<!-- http://www.nltk.org/install.html -->


import nltk 

nltk.download()

<!-- https://www.nltk.org/data.html -->

In [14]:
from nltk.corpus import wordnet as wn

Definition: synset = (from wiki) (information science) A set of one or more synonyms that are interchangeable in some context without changing the truth value of the proposition in which they are embedded.

2.3.2) Display sysents for "machine"

In [19]:
wn.synsets('machine')

[Synset('machine.n.01'),
 Synset('machine.n.02'),
 Synset('machine.n.03'),
 Synset('machine.n.04'),
 Synset('machine.n.05'),
 Synset('car.n.01'),
 Synset('machine.v.01'),
 Synset('machine.v.02')]

2.3.3) Display all definitions (.definition()) for synsets (machine)

In [24]:
#TODO
for synset in wn.synsets('machine'):
    print("* ", synset.definition())
    print("")

*  any mechanical or electrical device that transmits or modifies energy to perform or assist in the performance of human tasks

*  an efficient person

*  an intricate organization that accomplishes its goals efficiently

*  a device for overcoming resistance at one point by applying force at some other point

*  a group that controls the activities of a political party

*  a motor vehicle with four wheels; usually propelled by an internal combustion engine

*  turn, shape, mold, or otherwise finish by machinery

*  make by machinery



2.3.4) For each synset (machine), display its hypernym (a word with a broad meaning constituting a category into which words with more specific meanings fall; a superordinate. For example, colour is a hypernym of red).

In [25]:
#TODO
for synset in wn.synsets('machine'):
    print("* ", synset.hypernyms())
    print("")

*  [Synset('device.n.01')]

*  [Synset('person.n.01')]

*  [Synset('organization.n.01')]

*  [Synset('mechanical_device.n.01')]

*  [Synset('organization.n.01')]

*  [Synset('motor_vehicle.n.01')]

*  [Synset('shape.v.02')]

*  [Synset('produce.v.02')]



See: http://www.nltk.org/howto/wordnet.html
for more examples

cośtam
